In [ ]:
#Install all required library based on requirements.txt file
#add whatever library you wanted
%pip install -U -r D:/Repo/Spark-SQL/requirements.txt

In [1]:
# spark session and launch default SPARK UI to monitor with default port 4040
# http://localhost:4040/jobs/
from pyspark.sql import SparkSession
from concurrent.futures import ThreadPoolExecutor, as_completed

spark = SparkSession.builder \
    .master("local") \
        .appName("Spark-SQL") \
            .getOrCreate()

In [6]:
def read_and_query(file_path, table_name):

    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True, sep="|")

    # Register the DataFrame as a temporary view
    df.createOrReplaceTempView(table_name)
    
    # Use Spark SQL to query the DataFrame
    result = spark.sql(f"SELECT * FROM {table_name} WHERE Id = 1")

In [4]:
import pandas as pd

path = "../File/Config.csv"

#function csv to tuples [(a,a),(b,b)] here using lambda x:x for x in x
def read_csv_as_tuples(path):
    df = pd.read_csv(path, header=0, sep="|")
    list_of_tuples = [tuple(row) for row in df.values]
    return list_of_tuples

#function normal df for iteration using iterrows
def read_csv(path):
    df = pd.read_csv(path, header=0, sep="|")
    return df

In [7]:
# read from root path by using ../
files_and_tables = read_csv_as_tuples(path)

# Use ThreadPoolExecutor to run tasks in parallel
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(read_and_query, file, table) for file, table in files_and_tables]
    
    # Wait for all tasks to complete
    for future in as_completed(futures):
        future.result()

In [ ]:
result = spark.sql("""
                   SELECT A.Id AS Id_A, B.Id AS Id_B
                    FROM DQC A
                    INNER JOIN DQC2 B ON A.Id = B.Id WHERE A.Id = 1
                    """)
    
# Show the result|
result.show()